## Deploy model to Azure Container Instance (ACI)

### Setup environment

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
 
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

/anaconda/envs/py36/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Azure ML SDK Version:  1.0.39


### Register the model

In [8]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

model = Model.register(workspace = ws,
                       model_path ="sky.hdf5",
                       model_name = "sky",
                       tags = {"sky": "unet"},
                       description = "A model that kinda indicates pixels that belong to sky",)

first_workspace	westus2	azure_challenge	westus2
Registering model sky


### Retrieve the model

In [2]:
from azureml.core import Workspace
from azureml.core.model import Model
import os 
ws = Workspace.from_config()
model=Model(ws, 'sky')

model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), "sky.hdf5")

os.stat(file_path)

os.stat_result(st_mode=33279, st_ino=20, st_dev=50, st_nlink=1, st_uid=0, st_gid=0, st_size=372551468, st_atime=1558472305, st_mtime=1558472305, st_ctime=1558472305)

### If you are smart you would test your model here

### Create scoring scipt

In [9]:
%%writefile score.py
import json
import numpy as np
import os
from keras.models import load_model

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('sky')
    model = load_model(model_path)
    

def run(raw_data):
    try:
        data = np.array(json.loads(raw_data)['data'])
        # make prediction
        y_hat = model.predict(data)
        # you can return any data type as long as it is JSON-serializable
        return y_hat.tolist()
    except Exception as e:
        error = str(e)
        return error

Writing score.py


### Create environment

In [10]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("keras")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

### Create config

In [11]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=5, 
                                               tags={"data": "sky"}, 
                                               description='Predict sky pixels with keras')

### Deploy in ACI

In [12]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

CPU times: user 1.07 ms, sys: 63 µs, total: 1.14 ms
Wall time: 612 µs


In [13]:
%%time
#deploy webservice
service = Webservice.deploy_from_model(workspace=ws,
                                       name='sky-ai-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

print(service.scoring_uri)

Creating image

Image creation operation finished for image sky-ai-svc:1, operation "Succeeded"
Creating service
Running..................................................................................
SucceededACI service creation operation finished, operation "Succeeded"
http://e4032dad-019b-404d-b59c-650d146ce6f3.westus.azurecontainer.io/score
CPU times: user 3.31 s, sys: 217 ms, total: 3.53 s
Wall time: 14min 18s


In [8]:
#Run to kill the deployment
service.delete()

### Test the deployment

In [23]:
import json
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import skimage.transform as trans

fig=plt.figure(figsize=(256, 256))

sample_images = mpimg.imread(os.path.join(os.getcwd(),"test/10.jpg"))
fig.add_subplot(1, 3, 1)
plt.imshow(sample_images, vmin=0, vmax=1)


sample_images = trans.resize(sample_images,(256,256))

demo_images = mpimg.imread(os.path.join(os.getcwd(), "test/10_mask.jpg"))
fig.add_subplot(1, 3, 2)
plt.imshow(demo_images, vmin=0, vmax=1)

test_samples = json.dumps({"data": [sample_images.tolist()]})
test_samples = bytes(test_samples, encoding='utf8')


# predict using the deployed model
# service takes in an image of shape (1, 256,256,1) as a list in json
result = np.array(service.run(input_data=test_samples))
fig.add_subplot(1, 3, 3)
plt.imshow(result[0], vmin=0, vmax=1)

plt.show()